In [1]:
# All Import Statements Defined Here
# Note: Do not add to this list.
# All the dependencies you need, can be installed by running .
# ----------------

import sys
assert sys.version_info[0]==3

from gensim.models import KeyedVectors
from gensim.test.utils import datapath
import pprint
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 5]
import nltk
nltk.download('reuters')
from nltk.corpus import reuters
import numpy as np
import random
import scipy as sp
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import PCA

START_TOKEN = '<START>'
END_TOKEN = '<END>'
# ----------------

[nltk_data] Downloading package reuters to /Users/Sahil/nltk_data...
[nltk_data]   Package reuters is already up-to-date!


# Exploring Word Vectors (30 Points)
Word Vectors are often utilized as the fundamental component for downstream NLP tasks, e.g. question answering, text generation, etc. so it is important to build some intuitions as to their strengths and weaknesses. Here, you shall first explore word vectors, derived from co-occurrence matrices, and then those derived via word2vec. 

**Note on Terminology:** The terms "word vectors" and "word embeddings" are often used interchangeably. The term "embedding" refers to fact that we are encoding aspects of the word's meaning in a lower dimensional space. As Wikipedia states, "*conceptually it involves a mathematical embedding from a space with one dimension per word to a continuous vector space with a much lower dimension*" (1). Additionally throughout the Jupyter notebook we will connote vectors with "[ ... ]" notation. We provide references for the referenced material throughout the notebook, as denoted by the (...) notation.


## Part 1: Count-Based Word Vectors
Most word vector derivations and implementations are underscored by the following idea:

*You shall know a word by the company it keeps (Firth, J. R. 1957:11)* (2)

Many word vector implementations are driven by the idea that similar words, i.e. synonyms, will be used in similar contexts. As a result, similar words will often be spoken or written along with a shared subset of words, i.e. contexts. By examining these contexts, we can try to develop embeddings for our words. With this intuition in mind, many "old school" approaches to constructing word vectors relied on word counts. Here we elaborate upon one of those strategies, co-occurrence matrices (3,4):

### Co-Occurrence
Finally, we can take word-entity comparisons down to the word-word level, i.e. a co-occurrence matrix. With a fixed window of size $n$, examine $w_i$ in a document. Now examine the $n$ preceding and subsequent words in that document, i.e. words $w_{i-n} \dots w_{i-1}$ and $w_{i+1} \dots w_{i+n}$ which fall inside the fixed window. Maintain a symmetric word-word matrix, M, where you update the count for entry M[i,j] if $w_j$ appears inside $w_i$'s window 

**Example: Co-Occurrence with Fixed Window of n=1**:

Document 1: "All that glitters isn't gold"

Document 2: "All's well that ends well"


|    *    | START | all | that | glitter | isn't | gold | all's | end | well | END |
|---------|-------|-----|------|---------|-------|------|-------|-----|------|-----|
| START   | 0     | 1   | 0    | 0       | 0     | 0    | 1     | 0   | 0    | 0   |
| all     | 1     |   0 | 1    | 0       | 0     | 0    | 0     | 0   | 0    | 0   |
| that    | 0     | 1   | 0    | 1       | 0     | 0    | 0     | 1   | 0    | 0   |
| glitter | 0     | 0   | 1    | 0       | 1     | 0    | 0     | 0   | 1    | 0   |
| isn't   | 0     | 0   | 0    | 1       | 0     | 1    | 0     | 0   | 0    | 0   |
| gold    | 0     | 0   | 0    | 0       | 1     | 0    | 0     | 0   | 0    | 1   |
| all's   | 1     | 0   | 0    | 0       | 0     | 0    | 0     | 0   | 1    | 0   |
| end     | 0     | 0   | 1    | 0       | 0     | 0    | 0     | 0   | 1    | 0   |
| well    | 0     | 0   | 1    | 0       | 0     | 0    | 1     | 1   | 0    | 1   |
| END     | 0     | 0   | 0    | 0       | 0     | 0    | 0     | 0   | 1    | 0   |

**Note**: In NLP, we often add START and END tokens to handle edge cases at the beginning and end of sentences, paragraphs, etc. so we imagine START and END tokens encapsulating each document here, i.e. "START All that glitters isn't gold END".

Now, we have word vectors rooted in word-word comparisons, but these vectors will be large (linear with number of distinct words in a corpus). Thus, our next step is to run dimension reduction, i.e. PCA (Principal Components Analysis) or SVD (Singular Value Decomposition) to select the top $k$ principal components, e.g. $U_k$ in the image below, as our k-dimensional word vectors.

For reference, here's a visualization of dimensionality reduction with SVD:
![alt text](./imgs/svd.png "SVD")

This reduced-dimensionality-co-occurrence representation helps us preserve semantic relationships between words, e.g. doctor and hospital will be closer than doctor and dog. In practice, this method is not tractable for large corpora because of the memory needed to perform PCA or SVD.

**Note**: If you're curious and want to learn more about PCA or SVD feel free to checkout (5 - 7). These course notes from CS168 provide a great high-level treatment of these general purpose algorithms. Though, for the purpose of this class, you only need to know how to extract the k-dimensional embeddings by utilizing pre-programmed implementations of either algorithm from the numpy, scipy, or sklearn python packages.

### Question 1: Plotting - Co-Occurrence, Word Embeddings [code] (10 Points)
1. Here, we will be using the reuters corpus. If you haven't run the import cell at the top of this page, please run it now. The corpus consists of 10,788 news documents totaling 1.3 million words. These documents span 90 categories and are split into train and test. For more details, please see https://www.nltk.org/book/ch02.html. We provide the read_corpus function that pulls out articles from the "coffee" category. 
3. Construct a method that constructs a co-occurrence matrix with a fixed window of 4, considering words 4 before and after the word upon which the window is centered.
4. Construct a method that performs dimension reduction on the matrix to produce k-dimensional embeddings, i.e. use PCA or SVD to take the top k-principal components and produce a new matrix of k-dimensional embeddings.
5. Compute the co-occurrence matrix with fixed window of 4, over the corpus. Then compute k=2, 2-dimensional embeddings.
6. Plot the 2D embeddings for `["japan", "america", "mexico", "canada", "coffee", "tea"]`

**Note**: We have provided a `read_corpus` function below. It already adds START and END tokens to each of the documents. You do **not** have to lemmatize the strings in the documents or perform any other form of pre-processing.

In [2]:
def read_corpus(category="coffee"):
    """ Read files from the specified reuter's category.
        Params:
            category (string): category name
        Return:
            list of lists, with words from each of the processed files
    """
    files = reuters.fileids(category)
    return [[START_TOKEN] + [w.lower() for w in list(reuters.words(f))] + [END_TOKEN] for f in files]


In [3]:
def distinct_words(corpus):
    """ Determine a list of distinct words for the corpus.
        Params:
            corpus (list of list of strings): corpus of documents
        Return:
            list of distinct words, number of distinct words
    """
    # ------------------
    # Your implementation here.
    
    # ------------------

In [4]:
def compute_co_occurrence_matrix(corpus):
    """ Compute co-occurrence matrix for the given corpus with a window size of 4.
        Params:
            corpus (list of list of strings): corpus of documents
        Return:
            numpy matrix (number of words, number of words) of co-occurrence counts,
            dictionary mapping row/col index -> word
    """
    # ------------------
    # Your implementation here.
    
    # ------------------

In [5]:
def reduce_to_k_dim(M, k=2):
    """ Reduce dimensionality of (m,n) matrix to (m,k) using dimension reduction techniques like PCA or SVD.
    
        Possible Dimension Reduction Functions:
            - http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html
            - http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html
    
        Params:
            M (numpy matrix): co-occurrence numpy matrix
            k (int): number of dimensions we want for dimension reduction
        Return:
            numpy matrix (m,k) -- our k-dim word embeddings
    """    

    # ------------------
    # Your implementation here.
    
    # ------------------

In [6]:
def plot_embeddings(M, word2Ind,
                    words=["japan", "america", "mexico", "canada", "coffee", "tea"]):
    """ Plot Embeddings of specified words (scatterplot).
        Params:
            M (numpy matrix -- (m, k)): embeddings
            word2Ind (map string -> int): word to index mapping
            words (list of strings): words whose embeddings we want to visualize
        Return:
            None
            
        Hint: Try using Matplotlib to make a scatterplot.
    """
    # ------------------
    # Your implementation here.
    
    
    # ------------------

In [ ]:
# Producing Your Plots -- Run Cell
corpus = read_corpus()
M, word2Ind = compute_co_occurrence_matrix(corpus)
M_hat = reduce_to_k_dim(M, k=2)
plot_embeddings(M_hat, word2Ind)

### Question 2: Plotting - Co-Occurrence, Word Embeddings [written] (2 points)
What do you notice in the plot? What clusters together in 2-dimensional embedding space? What doesn't cluster together?


## Part 2: Prediction-Based Word Vectors
As discussed in class, more recently prediction-based word vectors have come into fashion, e.g. Word2Vec. Here, we shall explore the embeddings produced by Word2Vec. Please make sure that you have downloaded the Word2Vec embeddings from https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit

Please revisit the class notes and lecture slides for more details on the word2vec algorithm. If you're feeling adventurous, challenge yourself and try reading the original paper (12).

### Question 2: Plotting -  Word2Vec, Word Embeddings [code]
Let's directly compare the word2Vec embeddings to those of the co-occurrence matrix.

1. Read the word2Vec vectors into memory.
2. Convert the (n, 300) dimension vector to (n,2) using the same dimension reduction techniques as earlier (SVD, PCA, etc.)
3. Plot the 2D embeddings for `["japan", "america", "mexico", "canada", "coffee", "tea"]`

**Note**: Running the `load_word2vec` function may take several minutes.

In [14]:
# Fill this variable with path to your downloaded and unzipped embeddings
# The file should be of format ".bin"
embeddings_fp = "/Users/Sahil/Desktop/cs224n-1819/assignments/prototypes/hw1/soln/embeddings/GoogleNews-vectors-negative300.bin"
def load_word2vec(words, embeddings_fp=embeddings_fp):
    """ Load Word2Vec Vectors
        Param:
            words (list of strings) - words that we must consider in vocabulary
            embeddings_fp (string) - path to .bin file of pretrained word vectors
        Return:
            numpy matrix (number of words, 300) of embeddings,
            map of row/col index -> word,
            KeyedVectors format of embeddings (doc https://radimrehurek.com/gensim/models/deprecated/keyedvectors.html)
    """
    embed_size = 300
    wv_from_bin = KeyedVectors.load_word2vec_format(datapath(embeddings_fp), binary=True)
    vocab = list(wv_from_bin.vocab.keys())
    word2Ind = {}
    M = []
    curInd = 0
    for w in words:
        try:
            M.append(wv_from_bin.word_vec(w))
            word2Ind[w] = curInd
            curInd += 1
        except KeyError:
            continue
    for w in np.random.choice(np.array(vocab).flatten(), 1000):
        try:
            M.append(wv_from_bin.word_vec(w))
            word2Ind[w] = curInd
            curInd += 1
        except KeyError:
            continue       
    return np.stack(M), word2Ind, wv_from_bin

In [15]:
## Run cell to load word vectors
## Note: This may take several minutes
words=["japan", "america", "mexico", "canada", "coffee", "tea"]
M, word2Ind, wv_from_bin = load_word2vec(words)
M_hat = reduce_to_k_dim(M, k=2)

In [ ]:
## Run cell to plot word vectors
plot_embeddings(M_hat, word2Ind, words)

### Question 2: Plotting -  Word2Vec, Word Embeddings [written] (2 points)
Are the clusters in this plot any different than before with the co-occurrence plot?


## Cosine Similarity
Now, that we have word vectors we need a way to quantify the similarity between individual words, according to these metric-spaces. One such metric is cosine-similarity. We will be using this in the following problem in order to find words that are "close" and "far" from one another.

We can think of n-dimensional vectors as points in n-dimensional space. If we take this perspective L1 and L2 Distances help quantify the amount of space "we must travel" to get between these two points. Another approach is to examine the angle between two vectors. From trigonometry we know that:

![alt text](./imgs/inner_product.png "Angle")

Instead of computing the actual angle, we can similarly leave the similarity in terms of $similarity = cos(\Theta)$. Formally the Cosine Similarity between two vectors $p$ and $q$ is defined as (10, 11):

$$Cosine\_Dist = \frac{p \cdot q}{||p|| ||q||}, \textrm{ where } Cosine\_Dist \in [-1, 1] $$ 

### Question 3: Synonyms & Antonyms - Word2Vec, Word Embeddings [code + written] (8 points)
1. Find a polysemous word (for example, "leaves" or "scoop") such that the top-10 most similar words contains related words from both meanings. For example, "leaves" has both "vanishes" and "stalks" in the top 10, and "scoop" has both "handed_waffle_cone" and "lowdown". You will probably need to try several polysemous words before you find one. Please state the polysemous word you discover and the various definitions that are surfaced in your exploration of similar words. Why do you think many of the polysemous words you tried didn't work? **(4 points)**

**Note**: The `wv_from_bin.most_similar(word)` functions may be helpful here. It ranks all other words with respect to their cosine similarity to a given word. Please see https://radimrehurek.com/gensim/models/keyedvectors.html#module-gensim.models.keyedvectors for more documentation.

In [1]:
## Write code for polysemous word exploration here
## ----------------------------------------------

## ----------------------------------------------

**Write Answer Here (Polysemous Word Exploration)**


2. Find three words (w1,w2,w3) where w1 and w2 are synonyms and w1 and w3 are antonyms, but dist(w1,w3) < dist(w1,w2). For example, "qualified" is closer to "unqualified" than to "skilled". Can you give a possible explanation for why this happens? **(4 points)**

**Note**: The `wv_from_bin.distance(w1, w2)` function may be helpful here in order to compute the cosine distance between two words.

In [2]:
## Write code for snonym/antonym word exploration here
## ----------------------------------------------

## ----------------------------------------------

**Write Answer Here (Synonym/Antonym Word Exploration)**


### Question 4: Analogies - Word2Vec, Word Embeddings [code] (4 points)
Interestingly Word2Vec vectors have been shown to sometime be able to exhibit analogies. Please use the `most_similar(positive=[], negative=[])` function which finds words who are most similar to the words in the `positive` list and and most dissimilar from the words in the `negative` list.

1. For the analogy "man : king :: woman : x", what is x? In the cell below, we show you how to adapt this analogy into the `most_similar(positive=[], negative=[])` syntax. **(1 point)**

**Note:** Documentation on the `most_similar` function can be found at https://radimrehurek.com/gensim/models/keyedvectors.html#module-gensim.models.keyedvectors)

In [8]:
# Run this cell to answer the analogy -- man : king :: woman : x
pprint.pprint(wv_from_bin.most_similar(positive=['woman', 'king'], negative=['man']))

**Write X (According to Word Vectors) Here:**

2. Find an example of analogy that holds according to these vectors. State analogy solution, according to the word vectors. **(2 points)**

In [3]:
## Insert code to demonstrate working analogy here
## ----------------------------------------------

## ----------------------------------------------

**Write Analogy (According to Word Vectors) Here:**

3. Find an example of analogy that does not hold according to these vectors. State the intended analogy and analogy solution, according to the word vectors. **(1 point)**

In [4]:
## Insert code to demonstrate broken analogy here
## ----------------------------------------------

## ----------------------------------------------

**Write Intended Analogy & Compare with Closest Words (According to Word Vectors) Here:**

### Question 5: Bias - Word2Vec, Word Embeddings [code + witten] (4 points)
It's important to be cognizant of the biases implicit to our word embeddings.

1. Run the cell below, to examine which terms are most similar to "woman" and "boss" and most dissimilar from "man". What do you find? **(1 point)**

In [7]:
# Run this cell
# Here `positive` indicates the list of words to be similar to and `negative` indicates the list of words to be
# most dissimilar from.
pprint.pprint(wv_from_bin.most_similar(positive=['woman', 'boss'], negative=['man']))

**Explain Your Findings Here**


2. Use the `most_similar` function to find another case where bias is exhibited by the vectors. **(1 point)**

In [5]:
## Insert code to demonstrate bias
## ----------------------------------------------

## ----------------------------------------------

**Write Your Example of Bias Here:**


3. What might be leading to the development of these biases in the word vectors? **(2 points)**

**Write Your Answer Here:**

# References

[1] https://en.wikipedia.org/wiki/Word_embedding

[2] https://en.wikipedia.org/wiki/John_Rupert_Firth

[3] http://web.stanford.edu/class/cs124/lec/vectorsemantics.video.pdf

[4] https://medium.com/data-science-group-iitr/word-embedding-2d05d270b285

[5] https://web.stanford.edu/class/cs168/l/l7.pdf

[6] http://theory.stanford.edu/~tim/s15/l/l8.pdf

[7] https://web.stanford.edu/class/cs168/l/l9.pdf

[8] https://en.wikipedia.org/wiki/Taxicab_geometry

[9] https://en.wikipedia.org/wiki/Euclidean_distance

[10] https://en.wikipedia.org/wiki/Inner_product_space

[11] https://en.wikipedia.org/wiki/Cosine_similarity

[12] https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf
